In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import timm
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm  

from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

c:\Users\faiza\.conda\envs\sci_data\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Define transformations for the input data
# Adjust the size for InceptionResNet's input requirements
transform = transforms.Compose([
    transforms.Resize((299, 299)),  # Adjusting to InceptionResNet size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [3]:
# Load and transform datasets
train_data = ImageFolder(root=r'C:\Mine\Work\Uzi\Signature_Verification\train', transform=transform)
val_data = ImageFolder(root=r'C:\Mine\Work\Uzi\Signature_Verification\validation', transform=transform)
test_data = ImageFolder(root=r'C:\Mine\Work\Uzi\Signature_Verification\test', transform=transform)

# Create data loaders
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels, eps=0.001)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return F.relu(x, inplace=True)

class InceptionResnetA(nn.Module):
    def __init__(self, in_channels):
        super(InceptionResnetA, self).__init__()
        self.branch1x1 = BasicConv2d(in_channels, 32, kernel_size=1)

        self.branch5x5_1 = BasicConv2d(in_channels, 32, kernel_size=1)
        self.branch5x5_2 = BasicConv2d(32, 32, kernel_size=5, padding=2)

        self.branch3x3dbl_1 = BasicConv2d(in_channels, 32, kernel_size=1)
        self.branch3x3dbl_2 = BasicConv2d(32, 48, kernel_size=3, padding=1)
        self.branch3x3dbl_3 = BasicConv2d(48, 64, kernel_size=3, padding=1)

        self.conv2d = nn.Conv2d(128, in_channels, kernel_size=1)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        branch1x1 = self.branch1x1(x)

        branch5x5 = self.branch5x5_1(x)
        branch5x5 = self.branch5x5_2(branch5x5)

        branch3x3dbl = self.branch3x3dbl_1(x)
        branch3x3dbl = self.branch3x3dbl_2(branch3x3dbl)
        branch3x3dbl = self.branch3x3dbl_3(branch3x3dbl)

        outputs = [branch1x1, branch5x5, branch3x3dbl]
        outputs = torch.cat(outputs, 1)
        outputs = self.conv2d(outputs)
        outputs = self.relu(outputs + x)
        return outputs

# Similarly, define InceptionResnetB and InceptionResnetC for other blocks

class InceptionResNetV1(nn.Module):
    def __init__(self, num_classes=1000):
        super(InceptionResNetV1, self).__init__()
        # Define the stem of the network
        self.stem = nn.Sequential(
            BasicConv2d(3, 32, kernel_size=3, stride=2),
            BasicConv2d(32, 32, kernel_size=3),
            BasicConv2d(32, 64, kernel_size=3, padding=1),
            nn.MaxPool2d(3, stride=2),
            BasicConv2d(64, 80, kernel_size=1),
            BasicConv2d(80, 192, kernel_size=3),
            nn.MaxPool2d(3, stride=2)
        )

        # Define the Inception-ResNet blocks
        self.inception_resnet_a = InceptionResnetA(192)
        # Add more blocks here...

        # Define the classifier
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(192, num_classes)

    def forward(self, x):
        x = self.stem(x)
        x = self.inception_resnet_a(x)
        # Pass through additional blocks...

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc(x)
        return x

# Create the model instance
model = InceptionResNetV1(num_classes=len(train_data.classes))


class InceptionResnetB(nn.Module):
    def __init__(self, in_channels):
        super(InceptionResnetB, self).__init__()
        self.branch7x7 = nn.Sequential(
            BasicConv2d(in_channels, 128, kernel_size=1),
            BasicConv2d(128, 128, kernel_size=(1, 7), padding=(0, 3)),
            BasicConv2d(128, 192, kernel_size=(7, 1), padding=(3, 0))
        )

        self.conv2d = nn.Conv2d(192, in_channels, kernel_size=1)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        branch7x7 = self.branch7x7(x)

        outputs = self.conv2d(branch7x7)
        outputs = self.relu(outputs + x)
        return outputs

class InceptionResnetC(nn.Module):
    def __init__(self, in_channels):
        super(InceptionResnetC, self).__init__()
        self.branch3x3 = nn.Sequential(
            BasicConv2d(in_channels, 192, kernel_size=1),
            BasicConv2d(192, 224, kernel_size=(1, 3), padding=(0, 1)),
            BasicConv2d(224, 256, kernel_size=(3, 1), padding=(1, 0))
        )

        self.conv2d = nn.Conv2d(256, in_channels, kernel_size=1)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        branch3x3 = self.branch3x3(x)

        outputs = self.conv2d(branch3x3)
        outputs = self.relu(outputs + x)
        return outputs


In [5]:
class InceptionResNetV1(nn.Module):
    def __init__(self, num_classes=1000):
        super(InceptionResNetV1, self).__init__()
        # Define the stem of the network
        self.stem = nn.Sequential(
            BasicConv2d(3, 32, kernel_size=3, stride=2),
            BasicConv2d(32, 32, kernel_size=3),
            BasicConv2d(32, 64, kernel_size=3, padding=1),
            nn.MaxPool2d(3, stride=2),
            BasicConv2d(64, 80, kernel_size=1),
            BasicConv2d(80, 192, kernel_size=3),
            nn.MaxPool2d(3, stride=2)
        )

        # Define the Inception-ResNet blocks
        # Adjust the number of blocks based on your requirements
        self.inception_resnet_a = nn.Sequential(*[InceptionResnetA(192) for _ in range(5)])
        self.inception_resnet_b = nn.Sequential(*[InceptionResnetB(192) for _ in range(10)])
        self.inception_resnet_c = nn.Sequential(*[InceptionResnetC(192) for _ in range(5)])

        # Define the classifier
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(192, num_classes)

    def forward(self, x):
        x = self.stem(x)
        x = self.inception_resnet_a(x)
        x = self.inception_resnet_b(x)
        x = self.inception_resnet_c(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = self.fc(x)
        return x

# Create the model instance
model = InceptionResNetV1(num_classes=len(train_data.classes))


In [6]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [7]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [8]:
# Training loop
num_epochs = 5  # Set the number of epochs
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    with tqdm(train_loader, unit="batch") as tepoch:
        for images, labels in tepoch:
            tepoch.set_description(f"Epoch {epoch+1}")

            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            tepoch.set_postfix(loss=loss.item())

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}')

    # Validation loop
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad(), tqdm(val_loader, unit="batch") as vepoch:
        for images, labels in vepoch:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        vepoch.set_postfix(validation_accuracy=100 * correct / total)

    print(f'Accuracy of the model on the validation images: {100 * correct / total}%')

print('Finished Training')

Epoch 1: 100%|██████████| 225/225 [01:44<00:00,  2.16batch/s, loss=0.0513]


Epoch [1/5], Loss: 0.46823190251986185


100%|██████████| 48/48 [00:09<00:00,  5.12batch/s]


Accuracy of the model on the validation images: 79.8828125%


Epoch 2: 100%|██████████| 225/225 [01:39<00:00,  2.26batch/s, loss=0.00323]


...

Accuracy of the model on the validation images: 82.87760416666667%


Epoch 5: 100%|██████████| 225/225 [01:38<00:00,  2.29batch/s, loss=4.53] 


Epoch [5/5], Loss: 0.38268827733066346


100%|██████████| 48/48 [00:08<00:00,  5.42batch/s]

Accuracy of the model on the validation images: 83.85416666666667%
Finished Training


In [9]:
torch.save(model, '5epoch_inceptionresnetv2.pth')


In [10]:
from sklearn.metrics import confusion_matrix, accuracy_score
import torch
import torch.nn.functional as F
from tqdm import tqdm

def evaluate_model(model, data_loader):
    model.eval()  # Set the model to evaluation mode

    all_scores = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in tqdm(data_loader, desc='Evaluating', unit='batch'):
            inputs = inputs.to(device)  # Ensure data is on the correct device
            labels = labels.to(device)
            outputs = model(inputs)
            probabilities = F.softmax(outputs, dim=1)[:, 1]  # Assuming binary classification
            all_scores.extend(probabilities.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Convert scores to binary predictions using a threshold (e.g., 0.5)
    threshold = 0.5
    predictions = [1 if score > threshold else 0 for score in all_scores]

    # Confusion Matrix
    tn, fp, fn, tp = confusion_matrix(all_labels, predictions).ravel()

    # Calculate Accuracy
    acc = accuracy_score(all_labels, predictions)

    # Additional Calculations
    num_false_accepted = fp
    num_false_rejected = fn
    total_num_forged = fp + tn
    total_num_genuine = fn + tp

    return acc, num_false_accepted, num_false_rejected, total_num_forged, total_num_genuine

# Example usage
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)  # Replace 'model' with your InceptionResNetV2 model

# Assuming val_loader and test_loader are defined
val_acc, val_fa, val_fr, val_total_forged, val_total_genuine = evaluate_model(model, val_loader)
print(f'Validation: Accuracy: {val_acc:.4f}, False Accepted: {val_fa}, False Rejected: {val_fr}, Total Forged: {val_total_forged}, Total Genuine: {val_total_genuine}')

test_acc, test_fa, test_fr, test_total_forged, test_total_genuine = evaluate_model(model, test_loader)
print(f'Test: Accuracy: {test_acc:.4f}, False Accepted: {test_fa}, False Rejected: {test_fr}, Total Forged: {test_total_forged}, Total Genuine: {test_total_genuine}')


Evaluating: 100%|██████████| 48/48 [00:09<00:00,  5.10batch/s]


Validation: Accuracy: 0.8385, False Accepted: 108, False Rejected: 140, Total Forged: 1240, Total Genuine: 296


Evaluating: 100%|██████████| 49/49 [00:09<00:00,  5.06batch/s]

Test: Accuracy: 0.8562, False Accepted: 85, False Rejected: 136, Total Forged: 1249, Total Genuine: 288


In [11]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

def plot_confusion_matrix(true_labels, predictions, classes):
    conf_mat = confusion_matrix(true_labels, predictions)
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues', 
                xticklabels=classes, yticklabels=classes)
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

# Example usage
classes = ['Forged', 'Genuine']  # Change as per your class names
plot_confusion_matrix(all_labels, predictions, classes)


NameError: name 'all_labels' is not defined